## UAV fleet Simulation 

This is for testing propouses

In [1]:
import numpy as np

# Custom modules
from helpers import local2global, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC
from plan import Plan, State
from env import Enviroment

kill_processes()

## Create Plans

In [2]:
offsets = [  # east, north, up, heading
    (5, 5, 0, 90),
    (10, 0, 0, 45),
    (-5, -10, 0, 225),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),
]
n_vehicles = len(offsets)
side_lens = (5, 7, 4, 1, 2)  # must agree with n_vehicles
local_paths = [
    Plan.create_square_path(side_len=side_len, alt=5) for side_len in side_lens
]
plans = [Plan.basic(wps=path, alt=5, wp_margin=0.5) for path in local_paths]

## Visualization Parameters

In [3]:
homes = np.array([offset[:3] for offset in offsets])
global_paths = [
    local2global(np.array(path), home, pairwise=True)
    for path, home in zip(local_paths, homes)
]
colors = ["Green", "Blue", "Red", "Orange", "Yellow"]  # must agree with n_vehicles
waypoints = {
    f"waypoint_{i+1}": {"pos": global_paths[i], "color": colors[i]}
    for i in range(n_vehicles)
}
plot_3d_interactive(
    waypoints, title="Simulation Markers", expand=[0.2, 0.2, 0.6], ground=-0.05
)

## Choose Simulator

In [4]:
simulator = Simulator(name="NONE", offsets=offsets, plans=plans)

simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))

simulator = Gazebo(
    offsets=offsets,
    plans=plans,
    world_path="simulators/gazebo/worlds/runway.world",
    vehicle_models=n_vehicles * ["drone"],
    markers=waypoints,
)

Launch Simulator

In [5]:
uavs = simulator.launch()
env = Enviroment(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀
Vehicle 3 launched 🚀
Vehicle 4 launched 🚀
Vehicle 5 launched 🚀


## Execute Plan

In [7]:
while env.vehs:
    env.get_positions()
    for uav in env.vehs.copy():
        env.send_closest_pos(uav)
        if uav.plan.state == State.DONE:
            env.remove(uav)
        else:
            uav.act()
kill_processes()

Vehicle 5:📍 Distance to target: 1.75 m
Vehicle 3:📍 Distance to target: 1.86 m


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'